In [1]:
import datasets
train = datasets.load_dataset(
    "ought/raft", "neurips_impact_statement_risks", split="train"
)
validation = datasets.load_dataset(
    "ought/raft", "neurips_impact_statement_risks", split="test"
)

/root/miniconda3/envs/combined_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from raft_baselines.classifiers import ChatGPTClassifier, GPT3Classifier, SetFitClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
gpt_3_classifier = GPT3Classifier(
    train, config="neurips_impact_statement_risks"
)

In [4]:
chat_gpt_classifier = ChatGPTClassifier(
    train, config="neurips_impact_statement_risks", do_semantic_selection=True
)

In [5]:
set_fit_classifier = SetFitClassifier(
    train, config="neurips_impact_statement_risks", model_head=RandomForestClassifier
)


model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 426.35it/s]
***** Running training *****
  Num examples = 2000
  Num epochs = 1
  Total optimization steps = 125
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [01:31<00:00, 91.37s/it]


In [6]:
from setfit import SetFitModel, SetFitTrainer

setfit_model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-mpnet-base-v2", non_differentiable_model_head=RandomForestClassifier)

# setfit expects data with columns ['label', 'text'], instead of ['ID', 'Impact statement', 'Label', 'Paper link', 'Paper title']
setfit_train_data = train.map(
    lambda example:
        {'label': example['Label'], 
         'text': f"Paper title: {example['Paper title']}, Impact statement: {example['Impact statement']}"})

setfit_trainer = SetFitTrainer(
    setfit_model, setfit_train_data
)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [7]:
setfit_trainer.train()

Generating Training Pairs: 100%|██████████| 20/20 [00:00<00:00, 418.42it/s]
***** Running training *****
  Num examples = 2000
  Num epochs = 1
  Total optimization steps = 125
  Total train batch size = 16
Epoch: 100%|██████████| 1/1 [01:28<00:00, 88.72s/it]


In [15]:

# from raft_baselines.utils.tokenizers import GPTTokenizer
print(setfit_model.predict_proba(["Paper title: GNN research, Impact statement: test2"])[0].tolist())

tensor([0.9000, 0.1000], dtype=torch.float64)


In [10]:
print(chat_gpt_classifier.classify({"Paper title": "GNN research", "Impact statement": "test2"}))
print(gpt_3_classifier.classify({"Paper title": "GNN research", "Impact statement": "test2"}))
print(set_fit_classifier.classify({"Paper title": "GNN research", "Impact statement": "test2"}))


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


cuda
{"doesn't mention a harmful application": 1.0, 'mentions a harmful application': 0.0}
cuda
{"doesn't mention a harmful application": 0.834995074353834, 'mentions a harmful application': 0.16500492564616595}


TypeError: sum() received an invalid combination of arguments - got (out=NoneType, axis=NoneType, ), but expected one of:
 * (*, torch.dtype dtype)
      didn't match because some of the keywords were incorrect: out, axis
 * (tuple of ints dim, bool keepdim, *, torch.dtype dtype)
 * (tuple of names dim, bool keepdim, *, torch.dtype dtype)


In [9]:
# import torch
# torch.cuda.empty_cache()